Import libraries

In [29]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score

import hopsworks

Get model from model registry

In [30]:
project = hopsworks.login()

# Get the model registry
mr = project.get_model_registry()

2025-01-06 22:11:22,017 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-06 22:11:22,029 INFO: Initializing external client
2025-01-06 22:11:22,030 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 22:11:23,763 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079


In [31]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="news_propagation_model",
    version=4,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [32]:
# Initialize the model
model = xgb.XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

XGBClassifier(base_score='5.141206E-1', booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['float', 'float', 'int', 'int', 'float', 'float',
                             'float', 'int', 'int', 'float', 'float'],
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Get features to predict

In [33]:
fs = project.get_feature_store()

In [34]:
user_query_fg = fs.get_feature_group(
    name="user_query",
    version=1,
)

In [35]:
user_query = user_query_fg.select_all().read()
user_query

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


,news_id,average_followers,average_follows,repost_total,post_total,average_favorite,news_lifetime,average_time_difference,repost_percentage,retweet_percentage_1_hour,nb_users_10_hours,average_repost
0,macron friends trump,3352.933333,2092.233333,92,16,29.000000,3.591471e+07,0.0,0.851852,0.009259,1,5.796296
1,mbappe in italy,6693.857143,219.285714,2,6,2.142857,2.429441e+07,0.0,0.250000,0.125000,1,0.250000


In [36]:
selected_features = user_query_fg.select_except(["news_id"])

In [37]:
features_to_predict = selected_features.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 


In [38]:
# Use the retrieved XGBoost model to make predictions on the batch data
predictions = model.predict(features_to_predict.tail(1))

# Display the first five predictions
predictions

ValueError: feature_names mismatch: ['average_followers', 'average_follows', 'repost_total', 'post_total', 'repost_percentage', 'average_repost', 'average_favorite', 'news_lifetime', 'nb_users_10_hours', 'average_time_difference', 'retweet_percentage_1_hour'] ['average_followers', 'average_follows', 'repost_total', 'post_total', 'average_favorite', 'news_lifetime', 'average_time_difference', 'repost_percentage', 'retweet_percentage_1_hour', 'nb_users_10_hours', 'average_repost']